In [89]:
# import dependencies
import json
import requests
import pandas as pd
# from config import app_token

from api_keys import weather_api_key

In [73]:
# read in csv file for crimes from one year prior to present
pastyear_data = pd.read_csv("Data/crimes_pastyear_topresent.csv", encoding="utf-8")


In [74]:
# drop columns that will not be necessary for analysis
clean_pastyear = pastyear_data.drop(columns=['BLOCK', ' IUCR', ' SECONDARY DESCRIPTION',' LOCATION DESCRIPTION','ARREST',
                                             'DOMESTIC','FBI CD','X COORDINATE','Y COORDINATE','LOCATION'])

In [75]:
# rename columns for readability
clean_pastyear = clean_pastyear.rename(columns={'CASE#':'CASE #',
                                                'DATE  OF OCCURRENCE':'DATE OF OCCURRENCE',
                                                ' PRIMARY DESCRIPTION':'PRIMARY DESCRIPTION',
                                                'WARD':'DISTRICT #'}
                                      )

In [81]:
# filter the dataframe to display only homicide incidents
homicide_pastyear = clean_pastyear.loc[clean_pastyear['PRIMARY DESCRIPTION'] == 'HOMICIDE']

homicide_pastyear = homicide_pastyear.reset_index()

del homicide_pastyear['index']

# add current year
Year = 2020
homicide_pastyear['Year'] = Year

homicide_pastyear.head()

,CASE #,DATE OF OCCURRENCE,PRIMARY DESCRIPTION,BEAT,DISTRICT #,LATITUDE,LONGITUDE,Year
0,JD424989,11/08/2020 01:22:00 PM,HOMICIDE,831,17.0,41.761848,-87.702741,2020
1,JD448552,12/03/2020 02:45:00 PM,HOMICIDE,2212,19.0,41.679104,-87.681114,2020
2,JD240881,05/23/2020 11:53:00 PM,HOMICIDE,1422,26.0,41.910030,-87.713732,2020
3,JD455780,12/11/2020 04:50:00 AM,HOMICIDE,231,3.0,41.795683,-87.619251,2020
4,JD425287,11/08/2020 06:39:00 PM,HOMICIDE,612,17.0,41.751583,-87.650128,2020


In [82]:
# add 2011 crime data
crime_2016 = pd.read_csv("Data/Crimes2016.csv", encoding="utf-8") 

# remove extra columns
clean_2016 = crime_2016.drop(columns=['Block','IUCR','Location Description',
                                      'Arrest','Domestic','FBI Code','X Coordinate','Y Coordinate','Location'])

# rename so data is consistent
clean_2016 = clean_2016.rename(columns={'Case Number':'CASE #',
                                        'Date':'Date of Occurance',
                                        'Primary Type':'Primary Description',
                                        'Ward':'District #'}
                              )

# filter for homicide
homicide_2016 = clean_2016.loc[clean_2016['Primary Description'] == 'HOMICIDE']

homicide_2016 = homicide_2016.reset_index()

del homicide_2016['index']

In [83]:
# add 2011 crime data
crime_2011 = pd.read_csv("Data/Crimes2011.csv", encoding="utf-8") 

# remove extra columns
clean_2011 = crime_2011.drop(columns=['Block','IUCR','Location Description','Arrest',
                                      'Domestic','FBI Code','X Coordinate','Y Coordinate','Location'])

# rename so data is consistent
clean_2011 = clean_2011.rename(columns=
                                   {'Case Number':'CASE #',
                                    'Date':'Date of Occurance',
                                    'Primary Type':'Primary Description',
                                    'Ward':'District #'})

# filter for homicide
homicide_2011 = clean_2011.loc[clean_2011['Primary Description'] == 'HOMICIDE']

homicide_2011 = homicide_2011.reset_index()

del homicide_2011['index']

In [84]:
# merge dataframes
frames = [homicide_2011, homicide_2016, homicide_pastyear]

homicide_merge = pd.concat(frames)

# check to see all years merged into file
check = homicide_merge["Year"].value_counts()
check

2020    813
2016    788
2011    438
Name: Year, dtype: int64

In [85]:
# read in csv file for locations of police stations in Chicago
police_stations = pd.read_csv("Data/police_stations.csv")

In [86]:
# drop columns that will not be necessary for analysis
clean_police_stations = police_stations.drop(columns=['WEBSITE','PHONE','FAX','TTY',
                                                      'X COORDINATE','Y COORDINATE','LOCATION'])

In [91]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "Chicago"

# Build query URL
query_url = url + "appid=" + weather_api_key + "&q=" + city

In [92]:
# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()

# historical weather in this link is not free, looking for new source